In [ ]:
# pip install openai 
# pip install azure-search-documents==11.4.0b8

import openai
from openai.embeddings_utils import get_embedding
import json

# Azure OpenAI Credentials
OPENAI_API_KEY = "" 
OPENAI_RESOURCE_ENDPOINT = ""

# Azure Cognitive Search Credentials
ACS_ENDPOINT = ""
ACS_KEY = ""

openai.api_type = "azure"
openai.api_key = OPENAI_API_KEY
openai.api_base = OPENAI_RESOURCE_ENDPOINT
openai.api_version = "2023-07-01-preview"

In [ ]:
file_path = "hotels.json"

with open(file_path, 'r') as file:  
    text = file.read()  

hotels = json.loads(text)

for hotel in hotels:
    print(hotel['hotelname'])

    text_for_embedding = "Name: "+hotel['hotelname']+" " \
            +"Price min: "+str(hotel['pricemin'])+" euro max: "+str(hotel['pricemax'])+" euro " \
            +"Location: City: "+hotel['city']+" Country: "+hotel['country']+" " \
            +"Features of the hotel: "+hotel['features']+" " \
            +"Description: "+hotel['description']
    hotel['text'] = text_for_embedding
    hotel['embedding'] = get_embedding(text_for_embedding, engine = 'text-embedding-ada-002')

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.models import Vector

from azure.search.documents.indexes.models import (
        SearchIndex,
        SearchField,
        SearchFieldDataType,
        SimpleField,
        SearchableField,
        VectorSearch,
        HnswVectorSearchAlgorithmConfiguration,
        CorsOptions
)

index_name = "hotels1"

def get_index(name: str):

    fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True, filterable=True),
        SearchableField(name="hotelname", type=SearchFieldDataType.String,retrievable=True,searchable=True),
        SearchableField(name="country", type=SearchFieldDataType.String,retrievable=True,searchable=True),
        SearchableField(name="city", type=SearchFieldDataType.String,retrievable=True,searchable=True),
        SearchableField(name="location", type=SearchFieldDataType.String,retrievable=True,searchable=True),
        SearchableField(name="features", type=SearchFieldDataType.String,retrievable=True,searchable=True),
        SearchableField(name="description", type=SearchFieldDataType.String,retrievable=True,searchable=True),
        SearchableField(name="text", type=SearchFieldDataType.String,retrievable=True,searchable=True),
        SimpleField(name="pricemin", type=SearchFieldDataType.Int32,retrievable=True,searchable=True),
        SimpleField(name="pricemax", type=SearchFieldDataType.Int32,retrievable=True,searchable=True),
        SearchField(
            name="embedding",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            searchable=True,
            vector_search_dimensions=1536,
            vector_search_configuration="my-vector-config",
        ),
    ]

    cors_options = CorsOptions(allowed_origins=["*"], max_age_in_seconds=60)

    scoring_profiles = []

    vector_search = VectorSearch(
        algorithm_configurations=[HnswVectorSearchAlgorithmConfiguration(name="my-vector-config", kind="hnsw")]
    )

    return SearchIndex(name=name, fields=fields, scoring_profiles=scoring_profiles, vector_search=vector_search, cors_options=cors_options)

client = SearchIndexClient(ACS_ENDPOINT, AzureKeyCredential(ACS_KEY))

index = get_index(index_name)
client.delete_index(index_name)
client.create_index(index)

search_client = SearchClient(ACS_ENDPOINT, index_name, AzureKeyCredential(ACS_KEY))
search_client.upload_documents(documents=hotels)

In [ ]:
def simple_hybrid_search(query):
    search_client = SearchClient(ACS_ENDPOINT, index_name, AzureKeyCredential(ACS_KEY))
    vector = Vector(value=get_embedding(query, engine = 'text-embedding-ada-002'), k=3, fields="embedding")

    results = search_client.search(
        search_text="",
        vectors=[vector],
        select=["id", "hotelname","text","features"],
        top=5
    )
    
    return results

search_results = simple_hybrid_search(query = "Hotels with a private beach cost max 300 euro")

for result in search_results:
    print(result)


In [ ]:
def getAnswerFromOpenAI(query):

    # Search in the Azure Search Index
    azure_search_results = simple_hybrid_search(query)
    
    results_for_prompt=""
    for result in azure_search_results:
        results_for_prompt += f"Hotel: {result['text']}\n"
      
    system_message = f"""Assistant is a large language model designed to help users find Hotels
    You have access to an Azure Cognitive Search index with hundreds of hotels.

    This information returned from the search to anwser the users question
    {results_for_prompt}

    If you don't know the answer, just say that you don't know. Don't try to make up an answer.
    """
    
    response = openai.ChatCompletion.create(
        engine="gpt-4-0613", #"gpt-35-turbo",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": query},
        ]
    )

    return response['choices'][0]['message']['content']

result = getAnswerFromOpenAI("Hotels with a private beach cost max 300 euro")
print(result)